# Building a bigram and using 100YearOfSolitute as dataset 

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [4]:
with open("100YearOfSolitude.txt", 'r', encoding='utf-8') as f:
    text = f.read()

char = sorted(set(text))
vocab_size = len(char)

# print(char)
# print(vocab_size)

In [5]:
string_to_int = {ch:i for i, ch in enumerate(char)}
int_to_string = {i:ch for i, ch in enumerate(char)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:[int_to_string[i] for i in l]

data = torch.tensor(encode(text), dtype=torch.long)

# print(data[:50])

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[59, 56, 76,  1, 56, 73, 56, 65],
        [ 1, 71, 59, 69, 66, 72, 58, 59],
        [59,  1, 52,  1, 55, 52, 65, 58],
        [72, 54, 71, 70,  1, 57, 69, 66]])
targets:
tensor([[56, 76,  1, 56, 73, 56, 65,  1],
        [71, 59, 69, 66, 72, 58, 59,  1],
        [ 1, 52,  1, 55, 52, 65, 58, 56],
        [54, 71, 70,  1, 57, 69, 66, 64]])


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

['\n', '¬', 'A', 'T', 'f', 'R', 'g', '5', 'i', '’', 'L', 'T', 'Y', 'v', '}', '(', '„', 'C', '2', '¬', 'y', 'a', '0', 'S', ' ', 'X', '‘', '6', '”', ',', '}', '(', 'R', 'g', '4', 'z', 'K', '6', '3', 'h', 'p', 'q', 'n', 'l', 'Z', '?', 'D', ' ', 'W', '\n', '2', 'k', 'U', '7', '-', 'A', 'j', 'U', 't', '—', 'z', '4', '\n', '1', 'z', 'D', 'G', '0', 'O', ':', 'M', 'c', 'M', ';', 'U', '•', 'v', '-', '2', 'i', 'v', 'C', 'm', 'N', 'j', 'a', 'C', '.', 'l', 'R', 'r', 't', 'c', '\n', 'm', 'C', 'c', '}', '•', 'v', 'W', '\n', 'L', '”', '„', 'd', '‘', '(', '8', 'F', 'm', 'o', 'U', '7', 'I', '-', 'L', 'a', '.', 'G', '0', '1', 'l', 'Z', '?', 'I', 'X', 'V', '(', 'H', 'u', 'h', '”', '4', 'd', 'D', '”', 'A', ';', 't', 'p', 'X', 'Z', 'W', 't', '©', '2', 'J', '&', 'Q', 'o', '8', 'A', '^', 'n', '3', ',', '9', 'k', '2', 'k', '/', 'j', 'G', ')', '5', '9', 'P', 'o', '¬', 'C', '7', ' ', '©', 'w', '5', 'f', 'j', 'n', 'B', 'v', 'i', ')', ';', 'M', '}', ';', '©', 'Y', '7', '-', ':', 'v', 'm', 'W', 't', '!', '3', '“',

In [30]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.578, val loss: 2.558
step: 250, train loss: 2.584, val loss: 2.545
step: 500, train loss: 2.568, val loss: 2.533
step: 750, train loss: 2.563, val loss: 2.540
2.2157702445983887


In [12]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

['\n', 'R', 'U', 'g', 'i', 'm', 'G', 'N', 'T', 'b', 'R', '.', ';', 'W', '”', 'q', 'X', 'l', ':', 'F', 'H', 'T', 'S', 'J', '4', 'b', 'S', '—', ',', '9', 'J', '4', 'Z', 'K', 'i', 'r', '.', 'X', 'u', 'B', '¬', "'", 'z', '-', 'g', '4', 'A', 'm', 'N', '“', '_', '4', '?', 'Q', 'Q', 'A', '6', '/', 'R', 'g', 'Q', 'N', 'T', '/', '3', '_', 'D', '&', 'h', 'g', 'f', 'H', 'G', '&', '?', '0', 'w', 'G', 'N', 'q', 'D', 'F', 's', 'h', 'R', '/', 'Y', 'j', 'L', 'p', 'p', 'N', '„', '5', 'L', 'c', '2', 'k', 'g', 'W', '\n', 'B', '—', 'z', 'F', 'S', 'W', 'D', 'n', ':', 'N', 'F', 'W', '(', 'i', 'x', 'r', 'e', 'g', 'f', '\n', 'M', '6', '.', '“', '—', 'Z', '3', 'C', 'V', 'l', '‘', '}', 'z', '-', 'S', '•', 'D', 'F', 'm', ' ', '!', 'X', 'U', 'f', 'w', 'q', 'v', '!', '7', 'J', '¬', 'G', 'M', 'X', 'c', 'V', 'Y', '&', '‘', 'd', 'N', '(', 'n', 'T', '3', '4', 'A', 'Z', '!', '•', 'E', 'n', 'l', '3', 'C', 'h', 'J', '—', 'i', 'c', 'R', 'i', 'S', 'w', 'j', 'a', '!', ')', '9', '4', 'Q', '•', 'W', 'j', 'Q', 'Q', 't', '.', '

In [ ]:
just checkin 